Perform multisequence alignment

In [ ]:
for fname in *.FNA
do
base=${fname%.FNA*}
mafft --adjustdirection --thread 10 ./${base}.FNA \
> ${base}.fas; done

Quantify proportion of alignment gaps

In [ ]:
for fname in *.fas
do
base=${fname%.fas*}
grep -o '-' ${base}.fas | wc -l > ./gapcount/${base}.txt
wc -m ${base}.fas > ./sumcount/${base}.txt
done

Concatenate genes

In [ ]:
java -jar 'abioscripts.jar' \
--seqconcat \
--filenames *.fas \
--outputfile /path/to/outputs

Tree construction for concatenated dataset

In [ ]:
raxmlHPC-PTHREADS \
-s concat.phylip \
-n concat \
-m GTRGAMMAI -N 100 -T 16 -p 71051 -f a -x 72855 -o outgroup

Tree construction for individual gene trees

In [ ]:
# determine best substitution model for each locus
for fname in *.fas
do
base=${fname%.fas*}
java -jar jModelTest.jar \
-d ${base}.fas \
-f -i -g 4 -s 3 -AICc -a -tr 16 \
-n ${base} \
-o ${base}.out
done

# convert alignment to PHYLIP/NEXUS format
for fname in *.fas
do
base=${fname%.fas}
perl fasta2relaxedPhylip.pl \
-f ${base}.fas \
-o ${base}.phylip
done

# construct gene tree
PhyML-3.1_macOS-MountainLion \
-i gene.phylip -b 100 -m substitutionModel \
-v e -a e -s SPR --rand_start --n_rand_starts 20 --r_seed 2345 \
--no_memory_check

Root individual gene trees

In [ ]:
nw_reroot \
geneTree.phylip_phyml_tree.txt \
outgroup > gene-reroot.txt

Tree constructon for species tree

In [ ]:
# combine rooted gene trees into a single file and remove names of interior nodes
mpest controlFile

# multi-locus, site-only resampling of bootstrap trees
multilocus_bootstrap_new.sh \
100 \
path/PhyML/bootstrapTrees \
boot_trees.txt \
path/mpest/bootstrapTrees \
BS \
site \
-

Estimate divergence time

In [ ]:
# calculate gradient and Hessian matrix of the branch lengths
mcmctree mcmctree-outBV.ctl

# sample posterior distribution of times and rates
mcmctree mcmctree.ctl

# perform convergence diagnostics across runs (in R)

# mcmc sampling from prior with no data
mcmctree mcmctree-pr.ctl
